# 大语言模型预训练-微调技术之QLoRA

## 步骤1 导入相关包

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
import os

os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
os.environ['HF_HOME'] = '/root/autodl-tmp/cache/'

## 步骤2 加载数据集

In [ ]:
ds = load_dataset("llm-wizard/alpaca-gpt4-data-zh")
ds

In [ ]:
ds[:1]

## 步骤3 数据集预处理

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")
tokenizer

In [ ]:
def process_func(example):
    # 设置最大长度为256
    MAX_LENGTH = 256
    # 初始化输入ID、注意力掩码和标签列表
    input_ids, attention_mask, labels = [], [], []
    # 对指令和输入进行编码
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    # 对输出进行编码，并添加结束符
    response = tokenizer(example["output"] + tokenizer.eos_token)
    # 将指令和响应的输入ID拼接起来
    input_ids = instruction["input_ids"] + response["input_ids"]
    # 将指令和响应的注意力掩码拼接起来
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    # 将指令的标签设置为-100，表示不计算损失；将响应的输入ID作为标签
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    # 如果输入ID的长度超过最大长度，截断输入ID、注意力掩码和标签
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    # 返回处理后的数据
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

## 步骤4 创建模型

In [ ]:
import torch
##修改
# low_cpu_mem_usage=True: 这个参数设定为True意味着在模型加载时会尽可能地减少CPU内存的使用。
# torch_dtype=torch.half: 这个参数设置了模型中张量的数据类型为半精度浮点数，这可以减少内存占用和计算时间，但可能会牺牲一些精度。
# device_map="auto": 这个参数设置了模型应该在哪个设备上运行。“auto”意味着它将自动选择可用的设备，优先选择GPU，如果没有GPU则选择CPU。
# load_in_4bit=True: 这个参数设置为True意味着在模型加载时将使用4位量化，这可以进一步减少内存占用。
# bnb_4bit_compute_dtype=torch.half: 这个参数设置了在4位量化时的计算数据类型，这里设置为半精度浮点数。
# bnb_4bit_quant_type="nf4": 这个参数设置了4位量化的类型，"nf4"是一种特定的量化策略。
# bnb_4bit_use_double_quant=True: 这个参数设置为True意味着在4位量化时使用双重量化。
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh",
                                              torch_dtype=torch.half,
                                              low_cpu_mem_usage=True, 
                                              device_map="auto", 
                                              load_in_4bit=True,
                                              bnb_4bit_quant_type="nf4", 
                                              bnb_4bit_use_double_quant=True)
model.dtype

In [ ]:
# 查看参数，查看模型有哪些层，可以用于添加LoRA旁路
for name, parameter in model.named_parameters():
    print(name,parameter.dtype)

### 1、PEFT 步骤1 配置文件

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model
## ,target_modules=["query_key_value"],r=8
config = LoraConfig(task_type=TaskType.CAUSAL_LM,r=8, target_modules=['query_key_value'])
config

In [ ]:
# 在深度神经网络 [deep neural network] 训练时，需要对每个参数或权重 [parameter/weight] 计算其对损失函数 
# [loss function] 的梯度 [gradient]，从而进行反向传播 [back propagation] 和优化[optimization]。
# 默认情况下不会计算输入数据 [input data] 的梯度，即使它们在计算中起到了关键的作用。但是，在某些应用场景中，
# 例如图像生成 [image generation]、注意力机制 [attention mechanism] 等，需要计算输入数据的梯度。此时，
# 可以通过启用计算输入梯度的功能，对输入数据进行求导并利用其梯度信息进行优化。
# 作用: 启用该功能这对于在保持模型权重固定的同时微调适配器权重非常有用。

model.enable_input_require_grads()

### 2、PEFT 步骤2 创建模型

In [ ]:
model = get_peft_model(model, config)
model

In [ ]:
# 查看配置
config

## 步骤5 配置训练参数

In [ ]:
args = TrainingArguments(
    output_dir="/root/autodl-tmp/cache/finetuning/bloom-1b4-zh-qlora",  # 指定模型训练结果的输出目录
    per_device_train_batch_size=4,  # 设置每个设备（如GPU）在训练过程中的批次大小为4
    gradient_accumulation_steps=8,  # 指定梯度累积步数为8，即将多个批次的梯度累加后再进行一次参数更新
    logging_steps=20,  # 每20个步骤记录一次日志信息
    num_train_epochs=1,  # 指定训练的总轮数为1
    gradient_checkpointing=True,  # 启用梯度检查点技术，可以减少内存占用并加速训练过程
    optim="paged_adamw_32bit"  # 指定分页优化器为"paged_adamw_32bit"，这是一种针对低秩模型的优化算法
)

## 步骤6 创建训练器

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

## 步骤7 模型训练

In [ ]:
trainer.train()

## 步骤8 模型推理

In [ ]:
from peft import PeftModel
from transformers import pipeline

#加载基础模型
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh", low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")

#加载lora模型
p_model = PeftModel.from_pretrained(model=model, model_id="/root/autodl-tmp/cache/finetuning/bloom-1b4-zh-qlora/checkpoint-500")

#模型推理
pipe = pipeline("text-generation", model=p_model, tokenizer=tokenizer, device=0)
ipt = "Human: {}\n{}".format("如何写好一个简历？", "").strip() + "\n\nAssistant: "
pipe(ipt, max_length=500, do_sample=True, )